# Building RAG with Qwen2.5

In [ ]:
import json
import os
import shutil
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Step 0: Clear the Chroma database if it exists
persist_dir = "./chroma.db"
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# Step 1: Folder for JSON Files
input_folder = "Vietnam-Law-rag_json"

# Step 2: Load All JSON Files and Convert to LangChain Documents
documents = []
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        base_file_name = os.path.splitext(file_name)[0]  # Remove the extension for `file_id`
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Convert JSON data to LangChain Document objects
        documents.extend([
            Document(
                page_content=entry["text"],
                metadata={
                    "id": entry["id"],
                    "article": entry["article"],
                    "clause": entry["clause"],
                    "title": entry["title"],
                    "file_id": base_file_name
                }
            )
            for entry in data
        ])

print(f"Loaded {len(documents)} documents from {input_folder}.")

# Step 3: Initialize HuggingFace Embeddings
embeddings_model = HuggingFaceEmbeddings()

# Step 4: Create Chroma Vector Store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir
)

print("Chroma database created and saved at:", persist_dir)

# Test query
## RAG database builded on cloud servers, fetch them then run the below cell

The aim is to optimize the returned data after the query search before push into the LLM Models, below here use Qwen2.5 for example.

Just download the chroma.db, then symlink or put them in the current working git folder, then run the second cell.

### CELL 2

In [1]:
import torch
from langchain_chroma import Chroma  # Use the updated Chroma import
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load the Chroma Database
persist_dir = "./chroma.db"

# Initialize the embedding function
embeddings_model = HuggingFaceEmbeddings()

# Load the Chroma database with the embedding function
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embeddings_model
)

print("Chroma database loaded.")

# Step 2: Load Qwen Model
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 3: Set Device for GPU/CPU
device = 0 if torch.cuda.is_available() else -1

# Step 4: Create a Text-Generation Pipeline with GPU/CPU
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=device,
    clean_up_tokenization_spaces=True
)

# Wrap the pipeline for LangChain
hf = HuggingFacePipeline(pipeline=pipe)

print("Model and pipeline initialized.")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

Chroma database loaded.
Model and pipeline initialized.


### CELL 3

In [ ]:
import re
from langchain.chains import RetrievalQA

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Increase retrieval limit
retriever = vectorstore.as_retriever(search_kwargs={"k": 300})

qa_chain = RetrievalQA.from_chain_type(
    llm=hf,
    retriever=retriever,
    return_source_documents=True
)

# Query and retrieval
query = "Tai nạn giao thông đường thủy nội địa là gì?"
result = qa_chain({"query": query})

# Print the Result
print("Answer:", result["result"])

# Print the Source Documents
print("Source Documents:")
for doc in result["source_documents"]:
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Giao thông đường thủy nội địa, hàng hải;

Bảo vệ môi trường trong hoạt động giao thông vận tải đường thủy nội địa.

Chính phủ thống nhất quản lý nhà nước về giao thông đường thuỷ nội địa.

Giao lại tài sản cầm giữ khi nghĩa vụ đã được thực hiện.

Đánh giá về hệ thống đào tạo;

Chính phủ thống nhất quản lý nhà nước về đấu giá tài sản.

Chính phủ thống nhất quản lý nhà nước về giao thông đường bộ.

Thực hiện các hoạt động kết nối giao thương, tham gia hệ thống phân phối ở nước ngoài và tại Việt Nam;

Gia đình gốc là gia đình của những người có quan hệ huyết thống.

Giáo dục truyền thống đoàn kết, yêu n­ước, yêu con người và thiên nhiên;

Giấy chứng nhận đủ điều kiện buôn bán thuốc thú y bị tẩy xóa, sửa chữa nội dung;

Tay giơ thẳng đứng để báo hiệu cho người tham gia giao thông ở các hướng dừng lại;

Đánh giá thực trạn

### CELL 4

In [22]:
import re
from langchain.chains import RetrievalQA
import torch

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Increase retrieval limit
retriever = vectorstore.as_retriever(search_kwargs={"k": 250})

# Define a custom retriever function that intercepts and modifies the results
def custom_retriever(query, k=300):
    # Retrieve relevant documents from the retriever
    results = retriever.get_relevant_documents(query)
    
    modified_documents = []
    
    for doc in results:
        # Example of cleaning content: Removing unwanted words or patterns (adjust regex as needed)
        cleaned_content = re.sub(r"unwanted_pattern", "", doc.page_content)
        
        # Ensure that the content is relevant to the query (optional: further filtering logic)
        if 'thời hạn' in cleaned_content.lower():  # Assuming query context involves time limits like 'thời hạn'
            modified_doc = {
                "metadata": doc.metadata,
                "page_content": cleaned_content,
            }
            modified_documents.append(modified_doc)
    
    return modified_documents

# Define the instruction for the model
instruction = "Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

# Use the custom retriever to retrieve and modify documents
query = "Trẻ em là gì"
retrieved_docs = custom_retriever(query)

# Debug: Check if any documents are retrieved and modified
if not retrieved_docs:
    print("No relevant documents retrieved or filtered out.")
else:
    print(f"Retrieved {len(retrieved_docs)} documents.")

# Concatenate the instruction with the retrieved documents
input_to_llm = instruction + " " + " ".join([doc["page_content"] for doc in retrieved_docs])

# Check the input format before passing to the model
print("Input to LLM:", input_to_llm[:500])  # Print the first 500 characters for debugging

# Send the modified input to your LLM (e.g., Qwen2.5 model)
result_from_llm = hf(input_to_llm)  # Assuming 'hf' is your model callable
print("LLM Output:", result_from_llm)

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

No relevant documents retrieved or filtered out.
Input to LLM: Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
LLM Output: Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.  Question: What is a common cause for a person's anxiety? Options: - having a bad day - drinking too much coffee - feeling sad - not getting enough sleep
A:
Having a bad day is a common cause for a person's anxiety. This can lead to feelings of dread or worry about the future and may trigger symptoms such as rapid heartbeat, sweating, trembling, and other physical reactions. Drinking too much coffee can also contribute to anxiety by affecting the body's chemistry and causing irritability.
